In [121]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.dates
from datetime import datetime
import random
import numpy as np
from datetime import datetime
import pandas

In [198]:
data_dtypes = {'Store':int,'DayOfWeek':int,'Date':datetime,'Sales':float,'Customers':int,'Promo':int,
               'StateHoliday':str, 'SchoolHoliday':int}
store_dtypes = {'Store':int,'StoreType':str,'Assortment':str,'CompetitionDistance':str,
                'CompetitionOpenSinceMonth':str,'CompetitionOpenSinceYear':str,'Promo2':int,
                'Promo2SinceWeek':str,'Promo2SinceYear':str,'PromoInterval':list}
train_df = pd.read_csv('~/Development/kaggle/rossman/data/train.csv',
                        dtype=data_dtypes,
                        parse_dates='Date'
                        )
test_df = pd.read_csv('~/Development/kaggle/rossman/data/test.csv',
                       dtype=data_dtypes,
                       parse_dates='Date'
                       )
store_df = pd.read_csv('~/Development/kaggle/rossman/data/store.csv',
                        quotechar='"',
                        index_col=0,
                        dtype=store_dtypes
                        )
store_df = store_data.fillna('')

Use some of the training set to make a mini training set...

The test_df doesn't contain label information, so we need to make our own test data set from the training data

In [247]:
#sample 20% of the train_data dataframe
sample_stores = random.sample(store_df.index, len(store_df)/20)
dev_train_df = train_df[train_df['Store'].isin(sample_stores)]

# Remove the mini_train data from the train dataframe
train_df = train_df[~train_df['Store'].isin(sample_stores)]

# use 10% for a dev_test data set
sample_stores = random.sample(store_df.index, len(store_df)/10)
dev_test_df = train_df[train_df['Store'].isin(sample_stores)]
train_df = train_df[~train_df['Store'].isin(sample_stores)]

In [248]:
#extract NumPy arrays from the Pandas Dataframes and convert the features array
dev_train_labels = dev_train_df['Sales'].values
feature_columns = ['Store',
                   'DayOfWeek',
#                   'Date',
                   'Customers',
                   'Promo',
#                   'StateHoliday',
                   'SchoolHoliday'
                  ]

dev_train_features = dev_train_df.drop('Sales',1)[feature_columns].values

dev_test_labels   = dev_test_df['Sales'].values
dev_test_features = dev_test_df.drop('Sales',1)[feature_columns].values

In [249]:
test_data_mean = np.mean(dev_train_labels, dtype=np.int64)
dev_train_labels = dev_train_labels - test_data_mean
dev_test_labels = dev_test_labels - test_data_mean

In [250]:
from sklearn.neighbors import KNeighborsClassifier

neighbors = 10
clf = KNeighborsClassifier(neighbors)
clf.fit(dev_train_features, dev_train_labels)
accuracy = clf.score(dev_test_features, dev_test_labels)
print 'Accuracy = %.4f' %(accuracy)

Accuracy = 0.1693


In [251]:
from sklearn.linear_model import LinearRegression


In [252]:
clf = LinearRegression()
clf.fit(dev_train_features, dev_train_labels)
print ('Coefficients: \n', clf.coef_)
print ('Residual sum of squares: %.2f' % np.mean((clf.predict(dev_test_features) - dev_test_labels) **2))
print ('variance score: %.2f' % clf.score(dev_test_features, dev_test_labels))

('Coefficients: \n', array([  5.04203138e-01,  -4.99957468e+01,   8.45903139e+00,
         9.96562234e+02,   7.81632863e+00]))
Residual sum of squares: 2649164.14
variance score: 0.79
